In [24]:
import json
import datasets
import pandas as pd
data_path = "../eval/structured_reports.csv"
data = pd.read_csv(data_path)

original_reports = data["original_report"]
print("lodaded ", len(original_reports), " reports")
structured_reports = data["structured_report"]    
print("lodaded ", len(structured_reports), " reports")
# save the first 100 structured reports as a list in a json
structured_reports_mimic = structured_reports[:100].to_list()
original_reports_mimic = original_reports[:100].to_list()
with open("test_predictions_mimic.json", "w") as f:
    json.dump(structured_reports_mimic, f)
structured_reports_chex = structured_reports[-72:].to_list()
original_reports_chex = original_reports[-72:].to_list()
with open("test_predictions_chexpert.json", "w") as f:
    json.dump(structured_reports_chex, f)

test_dataset = datasets.load_dataset("StanfordAIMI/srrg_findings_impression", split="test_reviewed")
test_dataset_mimic = test_dataset.filter(lambda example: 'mimic' in example['id'])
test_dataset_chex = test_dataset.filter(lambda example: 'chexpert' in example['id'])

print("loaded ", len(test_dataset_mimic), " references")   
print("loaded ", len(test_dataset_chex), " references")
references_mimic = [example['original_report'] for example in test_dataset_mimic]
structured_references_mimic = [example['structured_report'] for example in test_dataset_mimic]
references_chex = [example['original_report'] for example in test_dataset_chex]
structured_references_chex = [example['structured_report'] for example in test_dataset_chex]

lodaded  233  reports
lodaded  233  reports
loaded  161  references
loaded  72  references


In [26]:
# check if the original reports are the same
number = 0
number2 = 0
for i in range(len(original_reports_mimic)):
    if original_reports_mimic[i] != references_mimic[i]:
        #print("original reports are not the same")
        #print(original_reports[i])
        #print(references_original[i])
        number += 1
        continue
    if structured_reports_mimic[i] != structured_references_mimic[i]:
        #print("structured reports are not the same")
        #print(structured_reports[i])
        #print(references_structured[i])
        number2 += 1
        continue
print("number of original reports that are not the same: ", number)
print("number of structured reports that are not the same: ", number2)

# save the first 100 structured reports as a list in a json
structured_reports_100 = structured_reports[:100].to_list()
with open("test_predictions_reports_mimic.json", "w") as f:
    json.dump(structured_reports_100, f)
structured_reports_chex = structured_reports[-72:].to_list()
with open("test_predictions_chexpert.json", "w") as f:
    json.dump(structured_reports_chex, f)


number of original reports that are not the same:  97
number of structured reports that are not the same:  3


In [19]:
print(structured_reports[0])

Exam Type: Upright AP view of the chest

History: Decreased respiratory rate after toxic ingestion.

Technique: Upright anteroposterior chest radiography.

Findings:
- Lungs and Airways:
  - Slightly hyperinflated lungs without focal consolidation.
- Pleura:
  - No large pleural effusion or pneumothorax.
- Cardiovascular:
  - Normal cardiac, mediastinal, and hilar contours.
- Tubes, Catheters, and Support Devices:
  - Multiple clips overlying the right breast and axillary region.

Impression:
1. No acute cardiopulmonary abnormality.


In [ ]:
from utils import get_lists
ref_data_path = "StanfordAIMI/srrg_findings_impression"
mistral = ["Exam Type: Chest radiograph, frontal and lateral views.\n\nHistory: Male patient with shortness of breath.\n\nTechnique: Frontal and lateral chest radiography.\n\nComparison: Not applicable.\n\nFindings:\nLungs and Airways:\n- Low lung volumes.\n\nPleura:\n- Bilateral loculated pleural effusions.\n\nCardiovascular:\n- Cardiac and mediastinal silhouettes are grossly stable.\n\nHila and Mediastinum:\n- Persistent elevation of the right hemidiaphragm.\n\nTubes, Catheters, and Support Devices:\n- Status post median sternotomy with superior two most wires fractured/deshiscence.\n\nMusculoskeletal and Chest Wall:\n- No specific findings reported.\n\nAbdominal:\n- No specific findings reported.\n\nOther:\n- Right base opacity, possibly due to atelectasis.\n\nImpression:\n1. Bilateral loculated pleural effusions.\n2. Right base opacity, which may represent atelectasis.\n3. Persistent elevation of the right hemidiaphragm.\n4. Status post median sternotomy with fractured/deshiscenced wires.<|end|>"]
bert = ["<|assistant|>Exam Type: Chest radiography, including frontal and lateral views.\n\nHistory: Male patient with shortness of breath.\n\nTechnique: Frontal and lateral chest radiography.\n\nComparison: Not applicable.\n\nFindings:\nLungs and Airways:\n- Low lung volumes.\n\nPleura:\n- Bilateral loculated pleural effusions.\n\nCardiovascular:\n- Cardiac silhouette appears grossly stable.\n\nHila and Mediastinum:\n- Mediastinal silhouette appears grossly stable.\n\nTubes, Catheters, and Support Devices:\n- Status post median sternotomy with superior two most wires fractured or deshiscence.\n\nMusculoskeletal and Chest Wall:\n- Persistent elevation of the right hemidiaphragm.\n\nImpression:\n1. Bilateral loculated pleural effusions.\n2. Right base opacity, possibly due to atelectasis.\n3. Persistent elevation of the right hemidiaphragm.\n4. Status post median sternotomy with fractured sternotomy wires."]
pred_file_mimic = "pred.json"
ref_findings_mimic_list, ref_impression_mimic_list, pred_findings_mimic_list, pred_impression_mimic_list = get_lists(pred_file_mimic, ref_data_path, dataset_name="mimic", split="test_reviewed")    
ref_findings_mimic_list = ["Findings:\n\nLungs and Airways:\n- Low lung volumes.\n\nPleura:\n- Bilateral loculated pleural effusions.\n\nCardiovascular:\n- Cardiac silhouette appears grossly stable.\n\nHila and Mediastinum:\n- Mediastinal silhouette appears grossly stable.\n\nTubes, Catheters, and Support Devices:\n- Status post median sternotomy with the superior two sternotomy wires fractured.\n\nMusculoskeletal and Chest Wall:\n- Persistent elevation of the right hemidiaphragm.\n- Right base opacity, suggestive of atelectasis.\n\n"]
ref_impression_mimic_list = ["Impression:\n1. Bilateral loculated pleural effusions.\n2. Right base opacity, likely representing atelectasis.\n3. Persistent elevation of the right hemidiaphragm.\n4. Fractured superior sternotomy wires."]
from StructEval.structeval.StructEval import StructEval
rr = StructEval(do_radgraph=True,
                    do_green=False,
                    do_bleu=True,   
                    do_rouge=True,
                    do_bertscore=True,
                    do_diseases=True)
    
results = rr.forward(refs=ref_findings_mimic_list, hyps=pred_findings_mimic_list, section="findings", aligned=False)
results2 = rr.forward(refs=ref_impression_mimic_list, hyps=pred_impression_mimic_list, section="impression", aligned=False)
print(results)
print(results2)

1 predictions loaded
161 references loaded
Number of predictions: 1
Number of references: 161
Truncating references to match number of predictions.


FileNotFoundError: [Errno 2] No such file or directory: '/Users/johannes/Library/Caches/radgraph/0.1.2/radgraph-xl.tar.gz'